In [1]:
from IPython.display import HTML

import math
import pandas as pd
import numpy as np

#from scipy.optimize import minimize
#from scipy.optimize import differential_evolution    

from lmfit import Minimizer, Parameters, report_fit

from bokeh.plotting import figure, output_notebook, show
from bokeh.models import LinearAxis, Range1d
from bokeh.models import DatetimeTickFormatter, MonthsTicker, NumeralTickFormatter, Legend


output_notebook()

Loading BokehJS ...

In [2]:
import sys
sys.path.insert(1,'../covid-website')

import SISV as s
from data import Data
from ContactRate import contact_rate

from maxlik import fit_model, loglik_leastsquare , loglik_poisson, loglik_negbin
from maxlik import piecewiseexp_diffevol

In [3]:
source = 'Johns Hopkins'    
region = 'US'
state = 'New York'

cutoff_positive = 1
cutoff_death = 1
truncate = 0


d = Data(source=source, region=region, state=state, county="", cutoff_positive=cutoff_positive, cutoff_death=cutoff_death, truncate=truncate) 


In [4]:
breaks = 5
expgrowth_params, expgrowth_breakpoints, likelihood, expgrowth_fit  = piecewiseexp_diffevol(d.x[d.minD+1:], d.dfatalities, breaks=breaks, minwindow=7)


../covid-website/maxlik.py:214: RuntimeWarning: overflow encountered in exp
  res[s] *= np.exp(ai * (x[s]-bi))
../covid-website/maxlik.py:92: RuntimeWarning: invalid value encountered in subtract
  negLL = -np.nansum( y*np.log(yhat)  - (y+r)*np.log(yhat+r) )  #removed constant terms to speed minimization <more sensitive to initial guess ???
../covid-website/maxlik.py:92: RuntimeWarning: divide by zero encountered in log
  negLL = -np.nansum( y*np.log(yhat)  - (y+r)*np.log(yhat+r) )  #removed constant terms to speed minimization <more sensitive to initial guess ???
/opt/conda/lib/python3.8/site-packages/scipy/optimize/_numdiff.py:497: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0
../covid-website/maxlik.py:214: RuntimeWarning: invalid value encountered in multiply
  res[s] *= np.exp(ai * (x[s]-bi))


In [5]:
def print_params(title, params):
    print('----------')
    print(title)
    print("detect:", params["detection_rate"])
    print("i0:", f'{params["i0"]:.0f}')
    print("R0:", f"{params['beta0']/params['gamma']:.1f}")
    for i in range(1, 1+params['segments']):
          print(f"{params['t{}'.format(i)]:.0f}",":", f"{params['beta{}'.format(i)]/params['gamma']:0.1f}")
            
def format_plot(p, legend):

    p.add_layout(Legend(items=legend, location='center'), 'right')

    p.yaxis[0].formatter = NumeralTickFormatter(format="0,0")

    p.xaxis.ticker = MonthsTicker(months=list(range(1,13)))

    p.xaxis.formatter=DatetimeTickFormatter(
            hours=["%d %B %Y"],
            days=["%d %B %Y"],
            months=["%d %B %Y"],
            years=["%d %B %Y"],
        )

    p.xgrid.ticker = p.xaxis.ticker

    p.xaxis.major_label_orientation = math.pi/4

    p.extra_x_ranges['x2'] = Range1d(d.x[0], d.x[-1])
    ax2 = LinearAxis(x_range_name="x2", axis_label="days")
    p.add_layout(ax2, 'below')

    p.ygrid.grid_line_color = 'navy'
    p.ygrid.grid_line_alpha = 0.3
    p.ygrid.minor_grid_line_color = 'navy'
    p.ygrid.minor_grid_line_alpha = 0.1

    p.yaxis[0].axis_label = 'Number of deaths per day'


In [6]:
exp_stages = 0
inf_stages = 1
crit_stages = 1
test_stages = 1
gamma_exp = 1/4
gamma = 1/4
gamma_crit = 1/14
gamma_pos = 1/14

#-------------
R0 = np.arange(0.8, 1.2, 0.01)
p = figure() #plot_width=800, plot_height=600)

r0s = []
doubling = []
checkr0s = []

for r0 in R0:
    try:
        dou = s.calc_doubling(exp_stages=exp_stages, inf_stages=inf_stages, beta=gamma*r0, gamma_exp=gamma_exp, gamma=gamma )
        r0s.append(r0)
        doubling.append(dou)

        #if r0>1:
        #    r00 = doubling_to_beta(doubling, exp_stages, inf_stages, gamma_exp, gamma)

        #    r00 = r00/gamma
        #    checkr0s.append(r00)
    except:
        pass


    p.line(r0s, doubling, line_width=3, line_color='black', line_dash='solid', alpha=0.3)
    #p.line(r0s, checkr0s, line_width=1, line_color=color, line_dash='solid', alpha=0.3)
    
show(p)
#-------------

params1 = {
    "population"    : d.population,
    "exp_stages"    : exp_stages,
    "inf_stages"    : inf_stages,
    "crit_stages"   : crit_stages,
    "test_stages"   : test_stages,
    "i0"            : 700,
    "death_rate"    : 0.73*1e-2,
    "detection_rate": 10e-2,
    "gamma_exp"     : gamma_exp,
    "gamma"         : gamma,
    "gamma_crit"    : gamma_crit,
    "gamma_pos"     : gamma_pos,
    "immun"         : 0,
    "vacc_start"    : 365,
    "vacc_rate"     : 0/365,  #30% of population per year
    "vacc_immun"    : 1/180,
    
    "interv"        : 'piecewise constant',
    "init_beta"     : '',
    'segments'      : breaks,
}

print('f0:', expgrowth_params[0])

doubling = math.log(2)/expgrowth_params[1]
beta0 = s.doubling_to_beta(doubling, exp_stages, inf_stages, gamma_exp, gamma)
print('doubling:', f'{doubling:.1f}', f'{beta0/gamma:.1f}')
params1['beta0'] = beta0

for i in range(len(expgrowth_breakpoints)):
    doubling = math.log(2)/expgrowth_params[i+2]
    
    betai = s.doubling_to_beta(doubling, exp_stages, inf_stages, gamma_exp, gamma)
    
    params1['t{}'.format(i+1)] = int(expgrowth_breakpoints[i])
    params1['beta{}'.format(i+1)] = betai

    print(f'{expgrowth_breakpoints[i]:.0f}', '\t', f'{doubling:.1f}', '\t', f'{betai/gamma:.1f}')
    
print_params("guess", params1)

params2 = params1.copy()

params2['i0']    = 10

params2['beta0'] = 2.35 * gamma
params2['beta1'] = 0.6 * gamma
params2['beta2'] = 1.15 * gamma
params2['beta3'] = 1.15 * gamma
params2['beta4'] = 1.5 * gamma
params2['beta5'] = 1.1 * gamma

    
params2['t1'] = 37
params2['t2'] = 80
params2['t3'] = 150
params2['t4'] = 230
params2['t5'] = 300

print_params("manual", params2)


y1 = s.SISV_J(d.x, params1)
y2 = s.SISV_J(d.x, params2)

f0: 0.7571143293642969
doubling: 2.2 2.5
32 	 -12.5 	 0.8
109 	 -121.4 	 1.0
206 	 7.6 	 1.4
251 	 -26.6 	 0.9
303 	 -61.3 	 1.0
----------
guess
detect: 0.1
i0: 700
R0: 2.5
31 : 0.8
109 : 1.0
206 : 1.4
251 : 0.9
303 : 1.0
----------
manual
detect: 0.1
i0: 10
R0: 2.4
37 : 0.6
80 : 1.1
150 : 1.1
230 : 1.5
300 : 1.1


In [7]:
p = figure(title='Daily Fatalities', plot_width=800, plot_height=600 , y_axis_type="log")
p.y_range.start = 1

#historical data
r0 = p.line(d.xd[d.minD+1:], d.dfatalities, line_width=1, line_color='red', line_dash='dotted', alpha=0.3)
r1 = p.circle(d.xd[d.minD+1:], d.dfatalities, size=5, color="red", alpha=0.3)

#plot 7-day rolling average
rolling = pd.DataFrame(data = d.dfatalities).interpolate().rolling(7).mean()
r2 = p.line(d.xd[d.minD+1:], rolling.loc[:,0].values, line_width=1, line_color='red')

#fit
r3 = p.line(d.xd[d.minD+1:], expgrowth_fit, line_width=1, line_color='black', line_dash='solid', alpha=1)

#sisv guess
r4 = p.line(d.xd[d.minD+1:], np.diff(y1[d.minD:,s.cF]), line_width=1, line_color='green', line_dash='solid', alpha=0.7)

#sisv manual
r5 = p.line(d.xd[d.minD+1:], np.diff(y2[d.minD:,s.cF]), line_width=1, line_color='blue', line_dash='solid', alpha=0.7)

legend = [
    ("COVID fatalities"   , [r0, r1]),
    ("COVID 7-day average"   , [r2]),
    
    ("expgrowth", [r3]),
    ("sisv guess", [r4]),
    ("sisv manual", [r5]),
]
format_plot(p, legend)
show(p)

#================

p = figure(title='Cumul  Fatalities', plot_width=800, plot_height=600 , y_axis_type="log")
p.y_range.start = 1

#historical data
r0 = p.line(d.xd[d.minD:], d.fatalities, line_width=1, line_color='red', line_dash='dotted', alpha=0.3)
r1 = p.circle(d.xd[d.minD:], d.fatalities, size=5, color="red", alpha=0.3)


#sisv guess
r4 = p.line(d.xd[d.minD:], y1[d.minD:,s.cF], line_width=1, line_color='green', line_dash='solid', alpha=0.7)

#sisv manual
r5 = p.line(d.xd[d.minD:], y2[d.minD:,s.cF], line_width=1, line_color='blue', line_dash='solid', alpha=0.7)

legend = [
    ("COVID fatalities"   , [r0, r1]),

    ("sisv guess", [r4]),
    ("sisv manual", [r5]),
]
format_plot(p, legend)
show(p)


In [8]:
#---------------------------------------------------------------------------------
#transform auxiliary values (which are bounded [0,1] into the breakpoint dates)
def aux_to_breakpoints(aux, x0, xn, minwindow):
    breakpoints = []
    bi1 = x0
    n=len(aux)
    for i,a in enumerate(aux):
        bi = a * (xn-(n-i)*minwindow - (bi1+minwindow)) + bi1+minwindow
        breakpoints.append(bi)
        bi1=bi   
    return breakpoints

#---------------------------------------------------------------------------------
def breakpoints_to_aux(breakpoints, x0, xn, minwindow):
    aux = []
    bi1 = x0
    n=len(breakpoints)
    for i,b in enumerate(breakpoints):
        temp = xn - (n-i)*minwindow - (bi1+minwindow)
        a = 1.0 if temp==0 else (b - bi1 - minwindow) / temp
        aux.append(a)
        bi1=b
    return aux  
    
#aux_to_breakpoints([0.5, 0.5], 0, 100, 10)


#---------------------------------------------------------------------------------


def override(key, overrides, default):
    return default if key not in overrides else overrides[key]

In [9]:
b = aux_to_breakpoints([0.5,0.5,0.5], 5, 100, 10)
a = breakpoints_to_aux(b, 5, 100, 10)
print(b)
print(a)

[42.5, 66.25, 83.125]
[0.5, 0.5, 0.5]


In [31]:
#---------------------------------------------------------------------------------
def SISV_lmfit(d, overrides={}, solver='leastsq'):

    #--------------------------------------
    def merge_params(params, constants, solve_ti=False):
    
        p = params.valuesdict()  #params should be a LMFIT Parameters instance; constants should be a dict
        for i, (k,v) in enumerate(constants.items()):
            if k not in p:  #do not override values that may already be in the Parameters array
                p[k]=v
    
        if solve_ti:  #we are solving for time breakpoints
            #calculate "ti" variables from "auxi" variables
            n = p['segments']
            
            aux = []
            for i in range(1, n+1):
                aux.append(params['aux{}'.format(i-1)])
                
            breakpoints = aux_to_breakpoints(aux, d.x[0], d.x[-1], minwindow)
            
            for i in range(1,n+1):
                #p['beta{}'.format(i)] = params[len(param_list)+i-1]
                p['t{}'.format(i)] = breakpoints[i-1]
                
        return p


    #--------------------------------------
    def lmfit_inner(params, x, constants, column, data=None):
        p = merge_params(params, constants, solve_ti=True if column==s.cF else False)   #solve for time breakpoints Ti through Auxi variables when using fatalities data
        yhat = s.SISV_J(x, p)
        if data is None:
            return yhat
        else:
            yy = np.diff(yhat[:,column], append=0)
            yy = yy - data
            return  np.nan_to_num(yy)



    #--------------------------------------
    #first stage: calibrate initial infectious population and contact rate over time on fatalities data
    #--------------------------------------
    
        
    gamma = override('gamma', overrides, 1/3)
    segments = override('segments', overrides, 7)
    minwindow = override('minwindow', overrides, 7)

    params = Parameters()
    #(name, value, vary, min, max, expr)
    params.add_many( 
                     ('exp_stages',    1, False),
                     ('inf_stages',    1, False),
                     ('crit_stages',   1, False),
                     ('test_stages',   1, False),
                     
                     ('gamma_exp',      gamma, False),
                     ('gamma',          gamma, False),
                     ('gamma_pos',      1/14, False),
                     ('gamma_crit',     1/14, False),
                     
                     ('death_rate',     0.5e-2, False),
                     ('detection_rate', 5e-2, False),

                     ('population',     d.population, False),

                     ('immun',          0, False),
                     ('vacc_start',     365, False),
                     ('vacc_rate',      0, False),
                     ('vacc_immun',     1/180, False),
        
                     ('segments',       segments         , False),

                     ('i0',             10        , True, 1, 1e6),

                     ('beta0',          2*gamma       , True, 0.05*gamma, 8*gamma),                 
                   )

    for i in range(1, segments+1):  
        params.add('aux{}'.format(i-1),value=0.1, vary=True, min=0, max=1)
        params.add('beta{}'.format(i), value= 0.7*gamma, vary=True,  min=0.05*gamma, max=8*gamma)

    #lmfit Parameters cannot accept string values so they get passed in a separate argument
    constants = { 
        'interv':'piecewise linear',
        'init_beta':'const',
    }

    for idx, (k,v) in enumerate(overrides.items()):
        if k in params:
            params[k].set(value=v)
        else:
            constants[k]=v
            
    fitter = Minimizer(lmfit_inner, params, fcn_args=(d.x[d.minD+1:], constants, s.cF, d.dfatalities))
    result = fitter.minimize(method=solver)

    p = merge_params(result.params, constants, solve_ti=True)  #merge the calibrated variables into the dictionary of params
    #print(p)    
    #-------------------------
    #second stage: calibrate detection rate on positive test results data
    #------------------------

    params = Parameters()
    #(name, value, vary, min, max, expr)
    params.add_many( 
                     ('detection_rate', 15e-2, True, 0, 1),
                   )

    fitter = Minimizer(lmfit_inner, params, fcn_args=(d.x, p, s.cP, d.positives))
    result = fitter.minimize(method=solver)

    p = merge_params(result.params, p, solve_ti=False) #merge the calibrated variables into the dictionary of params




    
    #yy = s.SISV(d.x, p)
    return p
    


In [34]:
%%time

p1 = SISV_lmfit(d, solver='nelder',
                  overrides={
                      'exp_stages'  : 2,
                      'inf_stages'  : 2,
                      'crit_stages' : 2,
                      'test_stages' : 2,
                      'death_rate'  : 0.73e-2,
                      'gamma_exp'   : 1/4,
                      'gamma'       : 1/4,
                      'gamma_crit'  : 1/21, #(17+21),
                      'gamma_pos'   : 1/(5+10),
                      'interv'      :'piecewise linear',
                      'init_beta'   : '',
                      'minwindow'   : 7,
                      'segments'    : 4,
                  })





CPU times: user 4.48 s, sys: 3.96 ms, total: 4.49 s
Wall time: 4.48 s


In [35]:
y1 = s.SISV_J(d.x, p1) 

def print_params(title, params):
    print('----------')
    print(title)
    print("detect:", params["detection_rate"])
    print("i0:", f'{params["i0"]:.0f}')
    print("R0:", f"{params['beta0']/params['gamma']:.1f}")
    for i in range(1, 1+params['segments']):
          print(f"{params['t{}'.format(i)]:.0f}",":", f"{params['beta{}'.format(i)]/params['gamma']:0.1f}")

print_params("p1", p1)
#print_params("p2", p2)
    
p = figure(title='Excess Fatalities', plot_width=800, plot_height=600 , y_axis_type="log")
p.y_range.start = 1

r0 = p.line(d.xd[d.minD+1:], d.dfatalities, line_width=1, line_color='red', line_dash='dotted', alpha=0.3)
r1 = p.circle(d.xd[d.minD+1:], d.dfatalities, size=5, color="red", alpha=0.3)

#plot 7-day rolling average
rolling = pd.DataFrame(data = d.dfatalities).interpolate().rolling(7).mean()
r2 = p.line(d.xd[d.minD+1:], rolling.loc[:,0].values, line_width=1, line_color='red')

r3_0 = p.line(d.xd[d.minD+1:], expgrowth_fit, line_width=1, line_color='blue', line_dash='solid', alpha=0.7)

r3_1 = p.line(d.xd[d.minD+1:], np.diff(y1[d.minD:,s.cF]), line_width=1, line_color='black', line_dash='solid', alpha=0.7)

#-------------

r0_p = p.line(d.xd[d.minP+1:], d.dpositives, line_width=1, line_color='red', line_dash='dotted', alpha=0.3)
r1_p = p.circle(d.xd[d.minP+1:], d.dpositives, size=5, color="red", alpha=0.3)

r3_1_p = p.line(d.xd[d.minP+1:], np.diff(y1[d.minP:,s.cP]), line_width=1, line_color='black', line_dash='solid', alpha=0.7)

legend = [
    ("COVID fatalities"   , [r0, r1]),
    ("COVID 7-day average"   , [r2]),
    
    ("expgrowth", [r3_0]),
    ("sisv", [r3_1]),

    ("COVID positives"   , [r0_p, r1_p]),
    ("p1_p", [r3_1_p]),
]

    
p.add_layout(Legend(items=legend, location='center'), 'right')
   
p.yaxis[0].formatter = NumeralTickFormatter(format="0,0")

p.xaxis.ticker = MonthsTicker(months=list(range(1,13)))

p.xaxis.formatter=DatetimeTickFormatter(
        hours=["%d %B %Y"],
        days=["%d %B %Y"],
        months=["%d %B %Y"],
        years=["%d %B %Y"],
    )

p.xgrid.ticker = p.xaxis.ticker

p.xaxis.major_label_orientation = math.pi/4


p.extra_x_ranges['x2'] = Range1d(d.x[0], d.x[-1])
ax2 = LinearAxis(x_range_name="x2", axis_label="days")
p.add_layout(ax2, 'below')


p.ygrid.grid_line_color = 'navy'
p.ygrid.grid_line_alpha = 0.3
p.ygrid.minor_grid_line_color = 'navy'
p.ygrid.minor_grid_line_alpha = 0.1

p.yaxis[0].axis_label = 'Number of deaths per day'

#p.toolbar_location = None
show(p)

rplot = figure(title='R0')
R0_1 = contact_rate(d.x, p1) / p1['gamma']
rplot.line(d.x, R0_1, line_width=1, line_color='black', line_dash='solid', alpha=1)
show(rplot)


----------
p1
detect: 2.220446049250313e-16
i0: 137
R0: 8.0
27 : 0.1
188 : 0.1
285 : 2.0
319 : 5.8
